# Homework
Create an RL that can balance pole in the CartPole game for 400 frames.

In [1]:
!apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
!pip install -q -U tf-agents-nightly pyvirtualdisplay gym[atari]

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [450 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 P

In [14]:
import gym

from tensorflow import keras
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [15]:
gym.envs.registry.all()

dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v3), EnvSpec(BipedalWalkerHardcore-v3), EnvSpec(CarRacing-v0), EnvSpec(Blackjack-v0), EnvSpec(KellyCoinflip-v0), EnvSpec(KellyCoinflipGeneralized-v0), EnvSpec(FrozenLake-v0), EnvSpec(FrozenLake8x8-v0), EnvSpec(CliffWalking-v0), EnvSpec(NChain-v0), EnvSpec(Roulette-v0), EnvSpec(Taxi-v3), EnvSpec(GuessingGame-v0), EnvSpec(HotterColder-v0), EnvSpec(Reacher-v2), EnvSpec(Pusher-v2), EnvSpec(Thrower-v2), EnvSpec(Striker-v2), EnvSpec(InvertedPendulum-v2), EnvSpec(InvertedDoublePendulum-v2), EnvSpec(HalfCheetah-v2), EnvSpec(HalfCheetah-v3), EnvSpec(Hopper-v2), EnvSpec(Hopper-v3), EnvSpec(Swimmer-v2), EnvSp

In [16]:
env = gym.make('CartPole-v0')
env._max_episode_steps = 400

In [17]:
# Look at the docstring.
?env.env

In [18]:
env.seed(42)
obs = env.reset()

obs

array([-0.01258566, -0.00156614,  0.04207708, -0.00180545])

In [19]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [ ]:
# Q-Table shape and Window size
# Qt_Size = [40] * len(env.observation_space.high)
# Window_Size = (env.observation_space.high - env.observation_space.low) / Qt_Size

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in subtract
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
#print(Qt_Size)
#print(Window_Size)

In [ ]:
#import numpy as np

# Put randum numbers in Q-Table
#Q_Table = np.random.uniform(low=0, high=4, size = (Qt_Size + [env.action_space.n]))

In [ ]:
# Converting continuous state to Q-Table indices
#def get_discrete_state(state): 
#  discrete_state = (state - env.observation_space.low) / Window_Size
#  return tuple(discrete_state.astype(np.int)) ### should be tuple !

In [21]:
#Q_Table.shape

In [22]:
#discrete_state = get_discrete_state(env.reset()) # returns only the initial state

#print("Starting State    :", discrete_state) ### position and velocity mapped into Q-Table row and col
#print("Starting Q-Values (per action 0, 1) :", Q_Table[discrete_state])
#print("Starting Action   :", np.argmax(Q_Table[discrete_state])) # action index -- > action itself 

In [24]:
import numpy as np 

# Hyperparameters
Learning_Rate = 0.1
Discount = 0.95
episodes = 60000

prior_reward = 0
Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

# New
epsilon = 1  # not a constant, going to be decayed. ; 50% from Q-table (Exploit - ArgMax) and 
epsilon_decay_value = 0.99995
#50%  Random actions ( Explore) 
#START_EPSILON_DECAYING = 1
#END_EPSILON_DECAYING = Step//2
#epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

In [25]:
# Put randum numbers in Q-Table
Q_Table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
Q_Table.shape

(30, 30, 50, 50, 2)

In [26]:
def get_discrete_state(state):
    discrete_state = state / np_array_win_size + np.array([15,10,1,10])
    return tuple(discrete_state.astype(np.int))

In [27]:
obs, reward, done, info = env.reset()
discrete_state = get_discrete_state(obs)

In [28]:
print(env.action_space.n)
print(env.observation_space.low)
print(env.observation_space.high)

2
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [29]:
import math
env.seed(42)
for episode in range(episodes):

  discrete_state = get_discrete_state(env.reset())
  done = False
  episode_reward = 0

  while not done:

    # NEW
    if np.random.random() > epsilon: # uniform random between 0 and 1 
      # Get action from Q table
      action = np.argmax(Q_Table[discrete_state])
    else:
      # Get random action
      action = np.random.randint(0, env.action_space.n)

   # action = np.argmax(q_table[discrete_state])
    obs, reward, done, info = env.step(action)
    episode_reward += reward
    new_discrete_state = get_discrete_state(obs)

    if not done:
      Q_Table[discrete_state + (action,)] = (1 - Learning_Rate) * Q_Table[discrete_state + (action,)] + Learning_Rate * (reward + Discount * np.max(Q_Table[new_discrete_state]))

    #elif done:
    #  Q_Table[discrete_state + (action,)] = -200

    discrete_state = new_discrete_state

  # NEW
  #if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
   #   epsilon -= epsilon_decay_value
  if epsilon > 0.05: 
    if episode_reward > prior_reward and episode > 10000:
      epsilon = math.pow(epsilon_decay_value, episode - 10000)

  prior_reward = episode_reward

In [30]:
#env.seed(42)

frames = []
#stop = 0

discrete_state = get_discrete_state(env.reset())
done=False

while not done:

    action=np.argmax(Q_Table[discrete_state])
    obs, reward, done, info = env.step(action)
    discrete_state = get_discrete_state(obs)

    img = env.render(mode="rgb_array")
    frames.append(img)

    # For the animation we keep printing frames for 50 steps
    #stop += done
    #if stop > 50:
    #  break

len(frames)

400

In [31]:
def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=20):
    fig = plt.figure()
    patch = plt.imshow(frames[399])
    plt.axis('off')
    anim = animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [32]:
plot_animation(frames)